## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

In [11]:
import spacy 
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


### Read reviews data

In [5]:
con=open("/content/drive/MyDrive/Colab Notebooks/samsung.txt",'r', encoding="utf-8")
samsung_reviews=con.read()
con.close()

In [ ]:
len(samsung_reviews.split("\n"))

46355

### Dataset is a text file where each review is in a new line

In [9]:
samsung_reviews.split("\n")[0:4]

["I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!",
 'nice phone, nice up grade from my pantach revue. Very clean set up and easy set up. never had an android phone but they are fantastic to say the least. perfect size for surfing and social media. great phone samsung',
 'Very pleased',
 'It works good but it goes slow sometimes but its a very good phone I love it']

### Will our hypothesis hold on real world data? `Product features---POS_NOUN`

In [12]:
review1=samsung_reviews.split("\n")[0]
review1=nlp(review1)

### Lets do nlp parse on part of one review in our dataset

In [13]:
for tok in review1[0:10]:
    print(tok.text,"---",tok.lemma_,"---",tok.pos_)

I --- I --- PRON
feel --- feel --- VERB
so --- so --- SCONJ
LUCKY --- lucky --- NOUN
to --- to --- PART
have --- have --- AUX
found --- find --- VERB
this --- this --- PRON
used --- use --- VERB
( --- ( --- PUNCT


#### Real world data is usually messy, observe the words `found` and `used`

In [16]:
pos = []
lemma = []
text = []
for tok in review1:
    pos.append(tok.pos_)
    lemma.append(tok.lemma_)
    text.append(tok.text)

In [18]:
nlp_table = pd.DataFrame({'text':text,'lemma':lemma,'pos':pos})
nlp_table.head()

,text,lemma,pos
0,I,I,PRON
1,feel,feel,VERB
2,so,so,SCONJ
3,LUCKY,lucky,NOUN
4,to,to,PART


In [19]:
## Get most frequent lemma forms of nouns
nlp_table[nlp_table['pos']=='NOUN']['lemma'].value_counts()

phone      3
lucky      1
line       1
one        1
year       1
upgrade    1
honesty    1
seller     1
Name: lemma, dtype: int64

#### It seems possible that if we extract all the nouns from the reviews and look at the top 5 most frequent lemmatised noun forms, we will be able to identify `What people are talking about?`

### Lets repeat this experiment on a larger set of reviews

In [ ]:
nouns = []
for review in samsung_reviews.split("\n")[0:100]:
    doc = nlp(review)
    for tok in doc:
        if tok.pos_=="NOUN":
            nouns.append(tok.lemma_.lower())

### Lets add some way of keeping track of time

In [20]:
from tqdm import tqdm
nouns = []
for review in tqdm(samsung_reviews.split("\n")[0:1000]):
    doc = nlp(review)
    for tok in doc:
        if tok.pos_=="NOUN":
            nouns.append(tok.lemma_.lower())
pd.Series(nouns).value_counts().head(5)

100%|██████████| 1000/1000 [00:13<00:00, 76.35it/s]


phone      1217
battery      92
time         92
screen       86
price        86
dtype: int64

In [ ]:
len(samsung_reviews.split("\n"))

46355

### Did you notice anything? What do you think will be the time taken to process all the reviews?

In [ ]:
(46355//1000)*17

782

In [ ]:
782//60

13

## Summary
- POS tag based rule seems to be working well
- We need to figure out a way to reduce the time taken to process reviews